In [18]:
import numpy as np
import scipy as sp
import scipy.io as sio
from scipy import ndimage
import scipy.stats
import matplotlib.pyplot as plt
from matplotlib_scalebar.scalebar import ScaleBar
import nipy
import nibabel #For various neuroimaging filetypes
from nibabel.analyze import load #load the image
from nipy import save_image
from nipy import load_image
from nipy.core.api import Image, vox2mni
import matlab.engine
import os.path

#from pCASL import mPLD_fitParkes, Parkes_fits, mPLD_getParams

%matplotlib inline
print("ac")

ac


In [25]:
f_base = '20170810_143430_ljoo_mouse_20170810_01_1_2_'
Mouse_ID = 'prac'
T2_num = '26'
CO2_num = '26'
fname = f_base+CO2_num+'.hdr'

In [22]:
CO2 = np.squeeze(np.array(nibabel.analyze.load(f_name).get_data()[:,:,:,:]))

In [7]:
dim1 = np.shape(CO2)[0]
dim2 = np.shape(CO2)[1]
nslices = 1
nframes = np.shape(CO2)[2]

### Masking data

In [8]:
# if a mask already exists,
if os.path.exists(Mouse_ID+'_m0_CO2.nii') == True:
    from nipy.labs.mask import compute_mask
    from nipy.core.api import Image, vox2mni
    
    m=0.35#35
    M=0.875#875
    PLD_masks = np.zeros([dim1,dim2])
    #for i in range(0,nPLDs):
    EPI_in = CO2
    mean_exp1 = np.zeros([dim1,dim2,1])#take the mean of all EPI's to generate 'mean_volume'
    mean_exp1[:,:,0]=np.mean(EPI_in,2)
    arr_img = Image(mean_exp1, vox2mni(np.eye(4)))#Image from array
    newimg = save_image(arr_img, Mouse_ID+'_m0_CO2.nii')
    mask_PLD_0000 = compute_mask(load_image(Mouse_ID+'_m0_CO2.nii').get_data(), reference_volume=None, m=m ,M=M, cc=False, opening=0, exclude_zeros=True)
    PLD_masks[:,:] = mask_PLD_0000[:,:,0]

In [9]:
# else; generate mask that encompasses grey and white matter and ventricles
if os.path.exists('pymask_sl1.mat') == False:
    eng = matlab.engine.start_matlab()
    eng.pymask(fname,1,'pymask_sl1.mat','analyze');
    eng.quit()

In [10]:
sl1_mask = sio.loadmat('pymask_sl1.mat')
PLD_masks = sl1_mask['mask'].T

[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]


In [11]:
# generate mask encompassing only cortical grey matter
if os.path.exists('pymask_GM2.mat') == False:
    eng = matlab.engine.start_matlab()
    eng.pymask(fname,1,'pymask_GM2.mat','analyze');
    eng.quit()


In [20]:
tt = np.array(sio.loadmat('pymask_GM2.mat')['mask'])
zz = np.zeros([dim1,dim2,1])
zz[:,:,0] = tt.T
vox2mni(np.eye(4))
arr_img = Image(zz, vox2mni(np.eye(4)))#Image from array
newimg = save_image(arr_img, 'GM_mask2.nii')

/home16/isjoo/anaconda3/envs/py34/lib/python3.4/site-packages/nipy/io/files.py:145: FutureWarning: Default `strict` currently False; this will change to True in a future version of nipy
  ni_img = nipy2nifti(img, data_dtype = io_dtype)


#### Pre-process the data. Motion correction + gaussian blurring.

In [28]:
En = CO2_num+'_CO2'
if os.path.exists('E'+En+'_afni') == False:
    !mkdir {'E'+En+'_afni'}
    
if os.path.exists('E'+En+'_afni/EPI_Image.nii') == False:
    sqimg = save_image(Image(load_image(fname), vox2mni(np.eye(5))), 'E'+En+'_afni/EPI_Image.nii')
    #this needs to be fixed so it works as bash command
    !2dImReg -input {'E'+En+'_afni/EPI_Image.nii'} -base 5 -prefix {'E'+En+'_afni/EPI_2dImReg'} -dprefix {'E'+En+'_afni/EPI_2dImReg_params'}

print("aa")


aa


In [ ]:


fig = plt.figure(figsize = (10,3))
motion_x = np.array(np.genfromtxt('E'+En+'_afni/EPI_2dImReg_params.dx'))
motion_y = np.array(np.genfromtxt('E'+En+'_afni/EPI_2dImReg_params.dy'))
motion_p = np.array(np.genfromtxt('E'+En+'_afni/EPI_2dImReg_params.psi'))
ax = fig.add_subplot(1,1,1)
plt.plot(motion_x[:,1])
plt.plot(motion_y[:,1])
plt.plot(motion_p[:,1])
plt.ylim([-2,2])
ax.set_xlabel('frame number')
ax.set_ylabel('N voxels of motion')
legend=plt.legend(['Motion in x','Motion in y','Rotational [degrees]'], fancybox=True,loc='best',framealpha=1)
legend.get_frame().set_facecolor('w')
for text in legend.get_texts():
    plt.setp(text, color = 'k', fontweight='bold',fontsize=12)

plt.show()